<a href="https://colab.research.google.com/github/finr/data-analysis-class/blob/master/PandasVsDask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install dask[complete]

### Coleta de dados
Cria os dados para a demonstração. Usando as viagens de táxi do mês de janeiro de 2017 da empresa Yellow Taxis de Nova Iorque, cria duas amostras de dados: uma com 1000 viagens e 88KBytes em disco e outra com aproximadamente 87 milhões de viagens e 7,2GBytes em disco.
O runtime do Colaboratory tem aproximadamente 12GBytes de memória RAM, o suficiente para que pandas carregue a primeira amostra de dados, mas não a segunda (runtime died).

In [0]:
!apt-get install time

In [0]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-01.csv -O large.csv

In [0]:
!time head -n1000 large.csv > small.csv
!time tail -n+3 large.csv > sample.csv
!time cat large.csv sample.csv sample.csv sample.csv sample.csv sample.csv sample.csv sample.csv sample.csv > very_large.csv

In [0]:
!ls -lah

**Usando os dados da [NYC Taxi and Limousine Commission (TLC)](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml) para o mês de janeiro de 2017, calcule a média de distância percorrida por viagem, em milhas, para diferentes quantidades de passageiros.**

### Usando pandas
**Limitando a análise a uma amostra de 1000 viagens, o carregamento e os cálculos são feitos.**

In [0]:
%%time
import pandas as pd
df = pd.read_csv('small.csv')

#print(df.groupby('passenger_count').trip_distance.mean())

In [0]:
df.head()

In [0]:
%%time
print(df.groupby('passenger_count').trip_distance.mean())

**Carregando todas as viagens, acontece um erro de memória.**

In [0]:
%%time
import pandas as pd
df = pd.read_csv('very_large.csv')

df.groupby('passenger_count').trip_distance.mean()


### **Limitação de pandas** 
O modelo de execução pressupõe que todos os dados serão carregados na memória ao mesmo tempo e que serão tratados por um único processo. Desse modo a memória RAM local é um limitante para o conjunto de dados que pode ser tratado.

### Usando dask
**Limitando a análise a uma amostra de 1000 viagens, o carregamento e a análise são feitos**

In [0]:
%%time
import dask.dataframe as dd
df = dd.read_csv('small.csv')
print(df.groupby('passenger_count').trip_distance.mean().compute())

**Para carregar todas as viagens, é preciso usar dois recursos do dask.dataframe:**


*   **blocksize:** indica o tamanho em bytes em que o arquivo será dividido durante o processamento, evitando carregar todos os dados ao mesmo tempo;
*   **dtype:** dicionário com os tipos de dados do dataframe que está sendo carregado, reduzindo o uso de memória ao não exigir que dask faça inferência sobre os tipos de dados.



In [0]:
%%time
# Aprox. 30MBytes
bytes_30M = 28 * 1024 * 1024

dtype_dictionary = {'VendorID':str, 'tpep_pickup_datetime':str, 'tpep_dropoff_datetime':str,
       'passenger_count':int, 'trip_distance':float, 'RatecodeID':int, 'store_and_fwd_flag':str,
       'PULocationID':int, 'DOLocationID':int, 'payment_type':int, 'fare_amount':float, 'extra':float,
       'mta_tax':float, 'tip_amount':float, 'tolls_amount':float, 'improvement_surcharge':float,
       'total_amount':float}


import dask.dataframe as dd

df = dd.read_csv('very_large.csv', blocksize=bytes_30M, dtype = dtype_dictionary)

print(df.groupby('passenger_count').trip_distance.mean().compute())